In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
# https://www.geeksforgeeks.org/how-to-calculate-mean-absolute-error-in-python/#
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.feature_selection import f_regression


df = pd.read_csv(r"C:\Users\jjap2\Poleto SCE\Poleto-SCE\dond_game_data.csv")

In [2]:
# Convert "Remaining Values" from string to list
df["Remaining Values"] = df["Remaining Values"].apply(ast.literal_eval)

# Extract all unique case values
all_values = set()
for values in df["Remaining Values"]:
    all_values.update(values)

# Create binary columns for each case value
for value in sorted(all_values):
    df[f"Case_{value}"] = df["Remaining Values"].apply(lambda x: 1 if value in x else 0)

In [3]:
display(df)

,Game ID,Round,Contestant Name,Contestant Gender,Contestant Race,Remaining Values,Board Value,Board Average,Board Balance,Previous Offer,...,Case_400000,Case_500000,Case_600000,Case_750000,Case_800000,Case_1000000,Case_1500000,Case_2000000,Case_3000000,Case_6000000
0,1,5,Tony,Male,White,"[25, 750, 5000, 50000, 200000, 300000]",555775,92629,0.506,94000.0,...,0,0,0,0,0,0,0,0,0,0
1,1,6,Tony,Male,White,"[25, 5000, 50000, 200000, 300000]",555025,111005,15.701,78000.0,...,0,0,0,0,0,0,0,0,0,0
2,1,7,Tony,Male,White,"[25, 5000, 200000, 300000]",505025,126256,14.287,105000.0,...,0,0,0,0,0,0,0,0,0,0
3,1,8,Tony,Male,White,"[25, 5000, 200000]",205025,68342,5.800,126000.0,...,0,0,0,0,0,0,0,0,0,0
4,1,9,Tony,Male,White,"[5000, 200000]",205000,102500,0.060,62000.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,107,2,Monica,Female,White,"[0, 1, 5, 25, 50, 75, 100, 200, 500, 50000, 75...",1725956,115064,1.276,9000.0,...,0,0,0,0,0,1,0,0,0,0
821,107,3,Monica,Female,White,"[1, 25, 50, 75, 100, 200, 500, 50000, 75000, 1...",425951,38723,0.316,10000.0,...,0,0,0,0,0,0,0,0,0,0
822,107,4,Monica,Female,White,"[1, 50, 75, 200, 500, 75000, 100000, 200000]",375826,46978,0.321,8700.0,...,0,0,0,0,0,0,0,0,0,0
823,107,5,Monica,Female,White,"[50, 75, 200, 75000, 100000, 200000]",375325,62554,0.816,21000.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# target variable
y = df["Offer"]

# Define features (all numerical + Case_X columns)
X = df.select_dtypes(include=[np.number]).drop(columns=["Offer", "Amount Won"])  # Exclude target

# how to F-Test
f_vals, p_vals = f_regression(X, y)

# Create a DataFrame to display results
f_test_res = pd.DataFrame({"Feature": X.columns, "F-Value": f_vals, "P-Value": p_vals})
f_test_res = f_test_res.sort_values(by="P-Value")

# feature results
display(f_test_res)

,Feature,F-Value,P-Value
3,Board Average,2627.191916,2.331593e-258
5,Previous Offer,2560.702033,7.024796e-255
9,Probability of Big Value,597.490755,1.230455e-99
8,Postgame,394.096415,5.803348e-72
6,Offer Percent of Average,276.877966,8.238631e-54
1,Round,181.287921,1.721321e-37
22,Case_400,85.938013,1.590600e-19
20,Case_200,71.609518,1.192415e-16
13,Case_5,63.388657,5.623509e-15
21,Case_300,63.224309,6.076491e-15


# Parameter Selection interpretation
1) Board Average
2) Previous Offer
3) Probability of Big Value
4) 

L1 Regulariazation (LASSO) - Thanks Dr Poulsen